In [15]:
import numpy as np
import pandas as pd
import networkx as nx
from pathlib import Path
from collections import Counter

In [16]:
taxa = ['B_caccae',
        'B_cellulosilyticus_WH2',
        'B_ovatus',
        'B_thetaiotaomicron',
        'B_uniformis',
        'B_vulgatus',
        'C_aerofaciens',
        'C_scindens',
        'C_spiroforme',
        # 'D_longicatena',
        'P_distasonis',
        'R_obeum'
        ]

order = ['B_cellulosilyticus_WH2', 
        'B_caccae', 
        'B_vulgatus', 
        'B_thetaiotaomicron', 
        'B_ovatus', 
        'R_obeum', 
        'B_uniformis', 
        'P_distasonis', 
        'C_scindens', 
        'C_aerofaciens', 
        'C_spiroforme', 
        # 'D_longicatena'
        ]

pretty_names_dir = {'B_caccae': '$B. caccae$',
                'B_cellulosilyticus_WH2': '$B. cellulosilyticus$',
                'B_ovatus': '$B. ovatus$',
                'B_thetaiotaomicron': '$B. thetaiotaomicron$',
                'B_uniformis': '$B. uniformis$',
                'B_vulgatus': '$B. vulgatus$',
                'C_aerofaciens': '$C. aerofaciens$',
                'C_scindens': '$C. scindens$',
                'C_spiroforme': '$C. spiroforme$',
                # 'D_longicatena': '$D. longicatena$',
                'P_distasonis': '$P. distasonis$',
                'R_obeum': '$R. obeum$'
                }

abbr_names_dir = {'B_caccae': 'Bca',
                'B_cellulosilyticus_WH2': 'Bce',
                'B_ovatus': 'Bov',
                'B_thetaiotaomicron': 'Bth',
                'B_uniformis': 'Bun',
                'B_vulgatus': 'Bvu',
                'C_aerofaciens': 'Cae',
                'C_scindens': 'Csc',
                'C_spiroforme': 'Csp',
                # 'D_longicatena': 'Dlo',
                'P_distasonis': 'Pdi',
                'R_obeum': 'Rob'
                }

pretty_names = [pretty_names_dir[taxa] for taxa in taxa]

pretty_abbrv_dir = {pretty_names_dir[organism]:abbr_names_dir[organism] for organism in order}

abbr_names = [abbr_names_dir[taxa] for taxa in taxa]

order_pretty = [pretty_names_dir[organism] for organism in order]

In [17]:
def evidence(x):
    if x < np.sqrt(10):
        return 'Weak'
    elif x < 10:
        return 'Moderate'
    elif x < 100:
        return 'Strong'
    else:
        return 'Very strong'
    
def interaction_type(x):
    if x > 0:
        return '+'
    elif x < 0:
        return '-'
    else:
        return np.nan

In [18]:
input_path = Path(f'mcnulty-results/best-chains/mcnulty-LF0-seed53/inference/posteriors')
output_path = Path(f'mcnulty-results/best-chains/mcnulty-LF0-seed53/inference/posteriors')

edges_lf0 = pd.read_table(f'{input_path}/interactions.tsv')
edges_lf0['interaction'] = edges_lf0['median'].apply(interaction_type)
edges_lf0['evidence'] = edges_lf0['bayes_factors'].apply(evidence)
edges_lf0['evidence'] = pd.Categorical(edges_lf0['evidence'], categories=['Moderate', 'Strong', 'Very strong'], ordered=True)
edges_lf0.dropna(inplace=True)
edges_lf0['Source'] = edges_lf0['Source'].map(abbr_names_dir)
edges_lf0['Target'] = edges_lf0['Target'].map(abbr_names_dir)
edges_lf0.set_index(['Source', 'Target'], inplace=True)
edges_lf0 = edges_lf0.loc[:,['median', 'interaction', 'evidence']]
edges_lf0['median'] = np.log10(np.abs(edges_lf0['median']))
edges_lf0.to_csv(f'{output_path}/interactions_network.tsv', sep='\t')
edges_lf0

median interaction  evidence
Source Target                                
Bca    Bov    -8.558691           +  Moderate
       Cae    -8.497826           -    Strong
       Csp    -8.517953           -    Strong
       Pdi    -8.574671           -  Moderate
Bce    Bov    -8.808798           +  Moderate
...                 ...         ...       ...
Rob    Bvu    -8.153779           -  Moderate
       Cae    -7.962109           -    Strong
       Csc    -7.949229           -    Strong
       Csp    -7.877683           -    Strong
       Pdi    -8.847131           -  Moderate

[84 rows x 3 columns]

In [19]:
Counter(edges_lf0['interaction'])

Counter({'-': 69, '+': 15})

In [20]:
input_path = Path(f'mcnulty-results/best-chains/mcnulty-HF0-seed3/inference/posteriors')
output_path = Path(f'mcnulty-results/best-chains/mcnulty-HF0-seed3/inference/posteriors')

edges_hf0 = pd.read_table(f'{input_path}/interactions.tsv')
edges_hf0['interaction'] = edges_hf0['median'].apply(interaction_type)
edges_hf0['evidence'] = edges_hf0['bayes_factors'].apply(evidence)
edges_hf0['evidence'] = pd.Categorical(edges_hf0['evidence'], categories=['Moderate', 'Strong', 'Very strong'], ordered=True)
edges_hf0.dropna(inplace=True)
edges_hf0['Source'] = edges_hf0['Source'].map(abbr_names_dir)
edges_hf0['Target'] = edges_hf0['Target'].map(abbr_names_dir)
edges_hf0.set_index(['Source', 'Target'], inplace=True)
edges_hf0 = edges_hf0.loc[:,['median', 'interaction', 'evidence']]
edges_hf0['median'] = np.log10(np.abs(edges_hf0['median']))
edges_hf0.to_csv(f'{output_path}/interactions_network.tsv', sep='\t')
edges_hf0

median interaction  evidence
Source Target                                
Bca    Bce    -8.477216           -  Moderate
       Bov    -8.369178           -    Strong
       Bth    -8.503201           -  Moderate
       Bvu    -8.362944           -    Strong
Bce    Cae    -8.671520           +    Strong
Bov    Cae    -8.293293           +    Strong
       Pdi    -8.299378           +  Moderate
       Rob    -8.199090           +  Moderate
Cae    Bov    -7.261454           -  Moderate
       Bth    -7.321448           +  Moderate
       Bvu    -7.253798           +  Moderate
       Csc    -7.231770           +  Moderate
Csc    Bov    -7.714540           -    Strong
Csp    Bca    -6.944469           -  Moderate
       Bov    -6.950741           -  Moderate
       Bth    -6.904661           -  Moderate
Pdi    Bvu    -7.964866           +    Strong
       Cae    -7.932077           +  Moderate
       Csc    -8.000123           +  Moderate
Rob    Bvu    -7.954928           +  Moderate
       Cae    -7.845929           +    Strong
       Pdi    -7.924031           +  Moderate

In [21]:
Counter(edges_hf0['interaction'])

Counter({'+': 13, '-': 9})